## Monitor agrakal daily data (ETFS) - Python

### Overview

This sample makes the inventory of the reference etfs for agrakal

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server
* *DailyBarsService*: to directly retrieve daily bars objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.topology.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*

***

# Run etf agrakal daily reference data sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas

In [ ]:
pip install systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import date
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_bars_pb2 as daily_bars
import systemathics.apis.services.daily.v1.daily_bars_pb2_grpc as daily_bars_service
import systemathics.apis.services.daily.v1.daily_prices_pb2 as daily_prices
import systemathics.apis.services.daily.v1.daily_prices_pb2_grpc as daily_prices_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *DailyBarsService* and *DailyPricesService*, we need to specify the **Instrument identifier**:

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
tickerexchange_array = [['HYG', 'XNYS'],
                        ['XLF', 'XNYS'],
                        ['XLU', 'XNYS'],
                        ['XLE', 'XNYS'],
                        ['IYT', 'BATS'],
                        ['IBB', 'XNMS'],
                        ['XLV', 'XNYS'],
                       ]
is_adjusted = False
length = len(tickerexchange_array)


#### 3.2 Define necessary methods
The following code snippets allows the user to generate a *DailyBarsRequest* or a *DailyPricesRequest* for the given ticker/exchange pair

In [ ]:
# define method to handle dailyprices request creation for each instrument
def get_dailyprices_request(ticker, exchange, is_adjusted):
    request = daily_bars.DailyBarsRequest(identifier = identifier.Identifier(exchange = exchange, ticker = ticker), adjustment = is_adjusted)
    return request

In [ ]:
# define method to handle dailyprices request creation for each instrument
def get_dailybars_request(ticker, exchange, is_adjusted):
    request = daily_prices.DailyPricesRequest(identifier = identifier.Identifier(exchange = exchange, ticker = ticker), adjustment = is_adjusted)
    return request

#### 3.3 Process daily requests
The following code snippet creates *gRPC client*, process daily bars and prices requests for each ticker/exchange pair:

In [ ]:
# process all topologies requests
credentials = grpc.ssl_channel_credentials()
prices, bars =[],[] # prepare storage for responses
today = datetime.today()
      
# iterate all instrument identifiers: exhange/ticker pairs
for i in range(length):
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:

            ticker = tickerexchange_array[i][0]
            exchange = tickerexchange_array[i][1]
            
            # --> prices
            prices_request = get_dailyprices_request(ticker, exchange, is_adjusted)
            prices_service = daily_prices_service.DailyPricesServiceStub(channel)

            # process the daily prices request
            prices_response = prices_service.DailyPrices(request=prices_request, metadata = [('authorization', token)])
            prices.append(prices_response)
            
            # --> bars
            bars_request = get_dailybars_request(ticker, exchange, is_adjusted)
            bars_service = daily_bars_service.DailyBarsServiceStub(channel)

            # process the daily bars request
            bars_response = bars_service.DailyBars(request=bars_request, metadata = [('authorization', token)])
            bars.append(bars_response)
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())

        
# get tick count data
print("Total asset requests: ", length)

Make sure we have as many responses for each data type than the number of inputs

In [ ]:
print("Total daily prices responses: ", len(prices))
print("Total daily bars responses: ", len(bars))

### Step 4: Export data
The 3 codes snippets below respectively export:
* a dashboard revealing for each instrument the **first date** and the **last date**
* **daily prices** (for each instrument)
* **daily bars** (for each instrument)

Export dashboard

In [ ]:
import csv


# process all daily responses
today = datetime.today()
filename = 'reference_agrakal_etfs_{0:%Y%m%d}.csv'.format(today)

with open(filename, mode='w') as dashboard_etfs_file:
    dashboard_etfs_writer = csv.writer(dashboard_etfs_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row
    dashboard_etfs_writer.writerow(['Ticker', 'Exchange','Type', 'First Date', 'Last Date'])
    
    # Prices
    for i in range(length):
        tpm_price = prices[i].data
        ticker = tickerexchange_array[i][0]
        exchange = tickerexchange_array[i][1]
        if len(tpm_price) > 0:
            first_price, last_price = tpm_price[0], tpm_price[-1]
            first_date = datetime(first_price.date.year, first_price.date.month, first_price.date.day)
            last_date = datetime(last_price.date.year, last_price.date.month, last_price.date.day)
            dashboard_etfs_writer.writerow([ticker,exchange, 'PRICE', '{0:%Y/%m/%d}'.format(first_date), '{0:%Y/%m/%d}'.format(last_date)])
        else: 
            print("Empty prices for {0}-{1}".format(ticker,exchange))
        
    # Bars
    for i in range(length):
        tpm_bar = bars[i].data   
        ticker = tickerexchange_array[i][0]
        exchange = tickerexchange_array[i][1]
        if len(tpm_bar) > 0:
            first_bar, last_bar = tpm_bar[0], tpm_bar[-1]
            first_date = datetime(first_bar.date.year, first_bar.date.month, first_bar.date.day)
            last_date = datetime(last_bar.date.year, last_bar.date.month, last_bar.date.day)
            dashboard_etfs_writer.writerow([ticker,exchange, 'BAR', '{0:%Y/%m/%d}'.format(first_date), '{0:%Y/%m/%d}'.format(last_date)])
        else: 
            print("Empty bars for {0}-{1}".format(ticker,exchange))

Export every prices in a csv

In [ ]:
suffix =''
if is_adjusted:
    suffix = '_adj_'

In [ ]:
for i in range(length):
    tpm_price = prices[i].data   
    my_dates=[datetime(p.date.year, p.date.month, p.date.day) for p in tpm_price]
    my_prices = [p.price for p in tpm_price]
    my_volumes = [p.volume for p in tpm_price]
    
    
    d = {'Date': my_dates, 'Price': my_prices, 'Volume': my_volumes } # create dictonary
    df = pd.DataFrame(data=d) # visualize dataframe
    
    filename = 'Export/Etf/{0}-{1}_dailyprices{2}.csv'.format(tickerexchange_array[i][0],tickerexchange_array[i][1],suffix)
    df.to_csv(filename, index=False)
    print("Exporting prices to {} ...".format(filename))

Export every bars in a csv

In [ ]:
for i in range(length):
    tpm_bar = bars[i].data   
    my_dates=[datetime(p.date.year, p.date.month, p.date.day) for p in tpm_bar]
    my_opens = [b.open for b in tpm_bar]
    my_highs = [b.high for b in tpm_bar]
    my_lows = [b.low for b in tpm_bar]
    my_closes = [b.close for b in tpm_bar]
    my_volumes = [b.volume for b in tpm_bar]
    
    
    d = {'Date': my_dates, 'Open': my_opens, 'High': my_highs, 'Low' : my_lows,'Close': my_closes, 'Volume': my_volumes }
    df = pd.DataFrame(data=d)
    
    filename = 'Export/Etf/{0}-{1}_dailybars{2}.csv'.format(tickerexchange_array[i][0],tickerexchange_array[i][1],suffix)
    df.to_csv(filename, index=False)
    print("Exporting bars to {} ...".format(filename))